In [31]:
# filter warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.models import model_from_json
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import cv2
import h5py
import os
import json
import datetime
import time

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [33]:
# load the user configs
with open('conf/conf.json') as f:    
    config = json.load(f)

# config variables
model_name    = config["model"]
weights     = config["weights"]
include_top   = config["include_top"]
train_path    = config["train_path"]
features_path   = config["features_path"]
labels_path   = config["labels_path"]
test_size     = config["test_size"]
results     = config["results"]
model_path    = config["model_path"]
seed          = config["seed"]

In [11]:
num_classes = 17

In [41]:
# start time
print("[STATUS] start time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))
start = time.time()

# create the pretrained models
# check for pretrained weight usage or not
# check for top layers to be included or not
image_size = (299, 299)

if model_name == "inceptionv3":
    base_model = InceptionV3(
        include_top=include_top, 
        weights=weights, 
        input_tensor=Input(shape=(299, 299, 3))
    )
    
    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

elif model_name == "xception":
    base_model = Xception(
        weights=weights
    )
    
    # add a global spatial average pooling layer
    x = base_model.get_layer('avg_pool').output
    
else:
    base_model = None
    
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer

predictions = Dense(num_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

print("[INFO] successfully loaded base model and model...")

# path to training dataset
train_labels = os.listdir(train_path)

print("train_labels:", train_labels)

# encode the labels
print("[INFO] encoding labels...")
le = LabelEncoder()
le.fit([tl for tl in train_labels])

# variables to hold features and labels
images = []
labels = []

# loop over all the labels in the folder
count = 1

for i, label in enumerate(train_labels):
    cur_path = train_path + "/" + label
    count = 1
    for image_path in glob.glob(cur_path + "/*.jpg"):
        img = image.load_img(image_path, target_size=image_size)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        
        images.append(x)
        labels.append(label)
        count += 1
    print("[INFO] completed image & label - " + label)

# encode the labels using LabelEncoder
le = LabelEncoder()
le_labels = le.fit_transform(labels)

# get the shape of training labels
print("[STATUS] training labels: {}".format(le_labels))
print("[STATUS] training labels shape: {}".format(le_labels.shape))

# end time
end = time.time()
print("[STATUS] end time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))

[STATUS] start time - 2019-05-14 22:40
[INFO] successfully loaded base model and model...
train_labels: ['snowdrop', 'lilyvalley', 'bluebell', 'daisy', 'cowslip', 'pansy', 'coltsfoot', 'fritillary', 'buttercup', 'daffodil', 'crocus', 'tulip', 'windflower', 'sunflower', 'tigerlily', 'iris', 'dandelion']
[INFO] encoding labels...
[INFO] completed image & label - snowdrop
[INFO] completed image & label - lilyvalley
[INFO] completed image & label - bluebell
[INFO] completed image & label - daisy
[INFO] completed image & label - cowslip
[INFO] completed image & label - pansy
[INFO] completed image & label - coltsfoot
[INFO] completed image & label - fritillary
[INFO] completed image & label - buttercup
[INFO] completed image & label - daffodil
[INFO] completed image & label - crocus
[INFO] completed image & label - tulip
[INFO] completed image & label - windflower
[INFO] completed image & label - sunflower
[INFO] completed image & label - tigerlily
[INFO] completed image & label - iris
[INF

In [42]:
images = np.array(images)
images = np.squeeze(images, axis=1)
labels = np.array(le_labels)
# verify the shape of features and labels
print("[INFO] images shape: {}".format(images.shape))
print("[INFO] labels shape: {}".format(labels.shape))

[INFO] images shape: (1360, 299, 299, 3)
[INFO] labels shape: (1360,)


In [43]:
#model.summary()

In [44]:
from tensorflow.keras.utils import to_categorical

print("[INFO] training started...")
# split the training and testing data
(x_train, x_test, y_train, y_test) = train_test_split(
    images,
    labels,
    test_size=test_size,
    random_state=seed
)

print(y_train)

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

print("[INFO] splitted train and test data...")
print("[INFO] train data  : {}".format(x_train.shape))
print("[INFO] test data   : {}".format(x_test.shape))
print("[INFO] train labels: {}".format(y_train.shape))
print("[INFO] test labels : {}".format(y_test.shape))

[INFO] training started...
[16  5 14 ...  8  3  3]
[INFO] splitted train and test data...
[INFO] train data  : (1224, 299, 299, 3)
[INFO] test data   : (136, 299, 299, 3)
[INFO] train labels: (1224, 17)
[INFO] test labels : (136, 17)


In [47]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler
import math

initial_lrate = 0.01

def decay(epoch, steps=100):
    drop = 0.96
    epochs_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
    return lrate

lr_sc = LearningRateScheduler(decay, verbose=1)

sgd = SGD(lr=initial_lrate, momentum=0.9, nesterov=True)

model.compile(
    loss='categorical_crossentropy',
    optimizer=sgd,
    metrics=['accuracy']
)

epochs = 35

history = model.fit(
    x=x_train,
    y=y_train,
    validation_data=(x_test, y_test),
    epochs=epochs, batch_size=256, callbacks=[lr_sc], verbose=1
)

Train on 1224 samples, validate on 136 samples

Epoch 00001: LearningRateScheduler reducing learning rate to 0.01.
Epoch 1/35
1224/1224 [==============================] - 13s 11ms/sample - loss: 2.7772 - accuracy: 0.1144 - val_loss: 2.7573 - val_accuracy: 0.1324

Epoch 00002: LearningRateScheduler reducing learning rate to 0.01.
Epoch 2/35
1224/1224 [==============================] - 4s 3ms/sample - loss: 2.4641 - accuracy: 0.3840 - val_loss: 2.5317 - val_accuracy: 0.1765

Epoch 00003: LearningRateScheduler reducing learning rate to 0.01.
Epoch 3/35
1224/1224 [==============================] - 4s 4ms/sample - loss: 2.0570 - accuracy: 0.6471 - val_loss: 2.2867 - val_accuracy: 0.3824

Epoch 00004: LearningRateScheduler reducing learning rate to 0.01.
Epoch 4/35
1224/1224 [==============================] - 4s 3ms/sample - loss: 1.6448 - accuracy: 0.8096 - val_loss: 2.1306 - val_accuracy: 0.3971

Epoch 00005: LearningRateScheduler reducing learning rate to 0.01.
Epoch 5/35
1224/1224 [=====